### Problem definition
### Put itens in a knapsack without exceed your capacity and obtaining maximum profit

In [ ]:
# Imports
from gurobipy import *
import numpy as np
import pandas as pd
import os
import time

In [2]:
# Read Data
def read_instance(path):
    with open(path) as r:
        num_itens = int(r.readline()) 
        r.readline() # empty line between data
        profits = [int(x) for x in r.readline().split()]
        r.readline()
        capacity = int(r.readline())
        r.readline()
        weights = [int(x) for x in r.readline().split()]

    return num_itens, profits, capacity, weights

### In this code we implement two algorithms
### Relaxed Dantzing, to obtain an upper bound, and
### Viable Dantzing, to obtain a lower bound.
### And a MIP model with gurobi api

In [3]:
def relaxed_dantzig(num_itens, profits, capacity, weights):
    '''
        Ordene em ordem não-crescente os itens de acordo com a relação pi/wi
        Insira itens na mochila tal forma que não exceda a capacidade.
        
        a solução relaxada não é inteira, é apenas [0, 1] logo
        x = [0, 0.4, 0, 1], por exemplo

        K é o conjunto de itens na mochila tal que
        o somatório para todo i pertencente a K, wi <= c
        
        definimos xh como item de parada, tal que
        (o somatório para todo i pertencente a K, wi) + wk > c 
        
        xk pertence a (0, 1)
    '''
    # values of itens is relation pi/wi
    values = [profits[i] / weights[i] for i in range(num_itens)]
    # Sort index in reversed order
    arg_sort_reversed = np.argsort(np.array(values))[::-1]

    # initialization of variables
    K = np.zeros(num_itens)
    x = np.zeros(num_itens)
    curr_capacity = 0

    # Fill Knapsack
    for i in arg_sort_reversed:
        # If knapsack has capacity to iten, put on
        if weights[i] + curr_capacity <= capacity:
            K[i] = 1
            x[i] = 1
            curr_capacity += weights[i]
        # Else, put on maximum possible 
        else:
            x[i] = (capacity - curr_capacity) / weights[i]
    # Calculate the cost of solution
    obj_value = x @ np.array(profits)
    return x, obj_value

In [4]:
def viable_dantzig(num_itens, profits, capacity, weights):
    '''
        Similar to relaxed dantzing, but the solution is binary,
        for garantee the viability.
    '''
    # Relaxed dantzig solution
    x, _ = relaxed_dantzig(num_itens, profits, capacity, weights) 
    
    # Transform all index in integer
    x = np.floor(x)
    # Calculate the cost of solution
    obj_value = x @ np.array(profits)
    return x, obj_value

In [5]:
def gurobi_knapsack(num_itens, profits, capacity, weights):
    # Create a model
    knap = Model()
    # Supress output log
    knap.setParam('OutputFlag', 0)
    # Add variables to model, relaxed [0, 1]
    x = knap.addVars(num_itens, lb=0, ub=1, name='x')
    # Add capacity constrain to model
    knap.addConstr((x.prod(weights)) <= capacity, name='knapsack')
    # Set a objective function to model, as Maximize the profit
    knap.setObjective(x.prod(profits), GRB.MAXIMIZE)
    # Solve the model
    knap.optimize()

    return knap.X, knap.ObjVal

### Go to tests!

In [6]:
num_itens, profits, capacity, weights = read_instance('data/instances_knapsack/10/10_100_1.txt')

In [7]:
x, obj_value = relaxed_dantzig(num_itens, profits, capacity, weights)
print('x :', list(x))
print('Objetive Function Value :', obj_value)

x : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.16666666666666666, 1.0, 1.0]
Objetive Function Value : 308.33333333333337


In [8]:
x, obj_value = viable_dantzig(num_itens, profits, capacity, weights)
print('x :', list(x))
print('Objetive Function Value :', obj_value)

x : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Objetive Function Value : 306.0


In [9]:
x, obj_value = gurobi_knapsack(num_itens, profits, capacity, weights)
print('x :', list(x))
print('Objetive Function Value :', obj_value)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\mareg\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-25
x : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.16666666666666666, 1.0, 1.0]
Objetive Function Value : 308.33333333333337


### Go to experiments with all instances!

In [10]:
def experiments_with(file_path, method):
    num_itens, profits, capacity, weights = read_instance(file_path)

    if method is 'relaxed_dantzig':
        x, obj_value = relaxed_dantzig(num_itens, profits, capacity, weights)
    elif method is 'viable_dantzig':
        x, obj_value = viable_dantzig(num_itens, profits, capacity, weights)
    elif method is 'gurobi_knapsack':
        x, obj_value = gurobi_knapsack(num_itens, profits, capacity, weights)
    else:
        print('Invalid Method')
        return False
    
    return x, obj_value

In [11]:
def run():
    folders = os.listdir('data/instances_knapsack')

    result = open(f'results/trabalho1/{time.time_ns()}.csv', 'w')
    result.write('instance_name,relaxed_cost,viable_cost,gurobi_cost')
    
    methods = ['relaxed_dantzig', 'viable_dantzig', 'gurobi_knapsack']
    
    for folder in folders:
        for instance in os.listdir(f'data/instances_knapsack/{folder}'):
            file_path = f'data/instances_knapsack/{folder}/{instance}'
            result.write('\n' + instance)
            for method in methods:
                _, cost = experiments_with(file_path, method)
                result.write(',' + str(cost))
    result.close()

In [12]:
run()

In [13]:
last_result = os.listdir(f'results/trabalho1/')[-1]
last_result_df = pd.read_csv(f'results/trabalho1/{last_result}')
last_result_df

,instance_name,relaxed_cost,viable_cost,gurobi_cost
0,10_100_1.txt,308.333333,306.0,308.333333
1,10_100_10.txt,302.160606,245.0,258.227273
2,10_100_2.txt,287.444444,287.0,287.444444
3,10_100_3.txt,244.236364,223.0,231.636364
4,10_100_4.txt,270.500000,269.0,270.500000
5,10_100_5.txt,241.529412,234.0,238.529412
6,10_100_6.txt,201.235294,197.0,201.235294
7,10_100_7.txt,316.520000,313.0,316.520000
8,10_100_8.txt,180.636364,163.0,170.500000
9,10_100_9.txt,235.640523,229.0,232.529412
